In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline


import pickle
import numpy as np

from SlideRunner_dataAccess.database import Database
from lib.calculate_F1 import *
from lib.extractDetections import *
optimal_threshold, F1_values = {},{}

Done importing database


# Object detection (single shot) based on RetinaNet

## First: Clustering- and Object-Detection-augmented Expert Labelling (CODAEL)

We first optimize the threshold on the train+validation set, and then run inference on the test set.

In [2]:
databasefile = 'databases/MITOS_WSI_CMC_CODAEL_TR_ROI.sqlite'

resfile = f'results/trainval_RetinaNet-CMC-CODAEL-512sh-b1.pth-CODAEL-val-inference_results_boxes.p.bz2'#
ident = f'CODAEL'

optimal_threshold[ident], F1scores, thrs = optimize_threshold(databasefile=databasefile, minthres=0.3, resfile=resfile)

resfile = f'results/test_RetinaNet-CMC-CODAEL-512sh-b1.pth-CODAEL-val-inference_results_boxes.p.bz2'#

F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, det_thres=optimal_threshold[ident])


Optimizing threshold for validation set of 14 files:  a8773be388e12df89edd.svs,460906c0b1fe17ea5354.svs,d0423ef9a648bb66a763.svs,50cf88e9a33df0c0c8f9.svs,da18e7b9846e9d38034c.svs,d7a8af121d7d4f3fbf01.svs,2191a7aa287ce1d5dbc0.svs,c4b95da36e32993289cb.svs,72c93e042d0171a61012.svs,4eee7b944ad5e46c60ce.svs,3d3d04eca056556b0b26.svs,084383c18b9060880e82.svs,d37ab62158945f22deed.svs,deb768e5efb9d1dcbc13.svs
Best threshold: F1= 0.7527488681131299 Threshold= 0.6300000000000003
Calculating F1 for test set of 7 files
Overall: 
TP: 3791 FP: 1318 FN:  1550 F1: 0.7255502392344497
Number of mitotic figures: 5341
Precision: 0.742 
Recall: 0.710


## Second: Object Detection-Augmented and Expert Labeled (ODAEL) data set variant

In this variant, an object-detection pipeline helped to find all mitotic figures and similar cells.

In [3]:
databasefile = 'databases/MITOS_WSI_CMC_ODAEL_TR.sqlite'

resfile = f'results/trainval_RetinaNet-CMC-ODAEL-512sh-b1.pth-ODAEL-val-inference_results_boxes.p.bz2'
ident = f'ODAEL'

optimal_threshold[ident], F1scores, thrs = optimize_threshold(databasefile=databasefile, minthres=0.3, resfile=resfile)

resfile = f'results/test_RetinaNet-CMC-ODAEL-512sh-b1.pth-ODAEL-val-inference_results_boxes.p.bz2'

F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, det_thres=optimal_threshold[ident])



Optimizing threshold for validation set of 14 files:  a8773be388e12df89edd.svs,460906c0b1fe17ea5354.svs,d0423ef9a648bb66a763.svs,50cf88e9a33df0c0c8f9.svs,da18e7b9846e9d38034c.svs,d7a8af121d7d4f3fbf01.svs,2191a7aa287ce1d5dbc0.svs,c4b95da36e32993289cb.svs,72c93e042d0171a61012.svs,4eee7b944ad5e46c60ce.svs,3d3d04eca056556b0b26.svs,084383c18b9060880e82.svs,d37ab62158945f22deed.svs,deb768e5efb9d1dcbc13.svs
Best threshold: F1= 0.7211228716060746 Threshold= 0.6700000000000004
Calculating F1 for test set of 7 files
Overall: 
TP: 3763 FP: 1478 FN:  1673 F1: 0.7048796478411539
Number of mitotic figures: 5436
Precision: 0.718 
Recall: 0.692


## Lastly: The initial manual labeling by two experts (MEL)

In [4]:
databasefile = 'databases/MITOS_WSI_CMC_MEL.sqlite'
resfile = f'results/trainval_RetinaNet-CMC-MEL-512sh-b1.pth-MEL-val-inference_results_boxes.p.bz2'#trainval_RetinaNet-CMC-MEL-512sh-b1.pth-MELshort-val-inference_results_boxes.p.bz2'
ident = f'MEL'

optimal_threshold[ident], F1scores, thrs = optimize_threshold(databasefile=databasefile, minthres=0.3, resfile=resfile)

resfile = f'results/test_RetinaNet-CMC-MEL-512sh-b1.pth-MEL-val-inference_results_boxes.p.bz2'

F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, det_thres=optimal_threshold[ident])



Optimizing threshold for validation set of 14 files:  a8773be388e12df89edd.svs,460906c0b1fe17ea5354.svs,d0423ef9a648bb66a763.svs,50cf88e9a33df0c0c8f9.svs,da18e7b9846e9d38034c.svs,d7a8af121d7d4f3fbf01.svs,2191a7aa287ce1d5dbc0.svs,c4b95da36e32993289cb.svs,72c93e042d0171a61012.svs,4eee7b944ad5e46c60ce.svs,3d3d04eca056556b0b26.svs,084383c18b9060880e82.svs,d37ab62158945f22deed.svs,deb768e5efb9d1dcbc13.svs
Best threshold: F1= 0.7036853821419737 Threshold= 0.6300000000000003
Calculating F1 for test set of 7 files
Overall: 
TP: 3325 FP: 1208 FN:  1869 F1: 0.6836640279634009
Number of mitotic figures: 5194
Precision: 0.734 
Recall: 0.640


# Two-stage approach
In this approach, we use the results of the previous RetinaNet as a first stage, and a ResNet-18 classifier as secondary stage. It is being evaluated for all sets.


### First, again, the CODAEL data set variant.

In [5]:
databasefile = 'databases/MITOS_WSI_CMC_CODAEL_TR_ROI.sqlite'

resfile = 'results/trainval_2ndstage_RetinaNet-CMC-CODAEL-512sh-b1.pth-CODAEL-val-inference_results_boxes.p.bz2'
ident = f'CODAEL_2nd'

optimal_threshold[ident], F1scores, thrs = optimize_threshold(databasefile=databasefile, minthres=0.3, resfile=resfile)

resfile = 'results/test_2ndstage_RetinaNet-CMC-CODAEL-512sh-b1.pth-CODAEL-val-inference_results_boxes.p.bz2'

F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, det_thres=optimal_threshold[ident])



Optimizing threshold for validation set of 14 files:  a8773be388e12df89edd.svs,460906c0b1fe17ea5354.svs,d0423ef9a648bb66a763.svs,50cf88e9a33df0c0c8f9.svs,da18e7b9846e9d38034c.svs,d7a8af121d7d4f3fbf01.svs,2191a7aa287ce1d5dbc0.svs,c4b95da36e32993289cb.svs,72c93e042d0171a61012.svs,4eee7b944ad5e46c60ce.svs,3d3d04eca056556b0b26.svs,084383c18b9060880e82.svs,d37ab62158945f22deed.svs,deb768e5efb9d1dcbc13.svs
Best threshold: F1= 0.8426933597159983 Threshold= 0.7500000000000004
Calculating F1 for test set of 7 files
Overall: 
TP: 4187 FP: 1027 FN:  1154 F1: 0.7933680720037897
Number of mitotic figures: 5341
Precision: 0.803 
Recall: 0.784


### Then, the ODAEL without any clustering help, detected by the dual stage setup:

In [6]:
databasefile = 'databases/MITOS_WSI_CMC_ODAEL_TR.sqlite'

resfile = 'results/trainval_2ndstage_CellClassifier_128px_ODAEL_b1.pthRetinaNet-CMC-ODAEL-512sh-b1.pth-ODAEL-val-inference_results_boxes.p.bz2'
ident = f'ODAEL_2nd'

optimal_threshold[ident], F1scores, thrs = optimize_threshold(databasefile=databasefile, minthres=0.3, resfile=resfile)

resfile = 'results/test_2ndstage_CellClassifier_128px_ODAEL_b1.pthRetinaNet-CMC-ODAEL-512sh-b1.pth-ODAEL-val-inference_results_boxes.p.bz2'

F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, det_thres=optimal_threshold[ident])



Optimizing threshold for validation set of 14 files:  a8773be388e12df89edd.svs,460906c0b1fe17ea5354.svs,d0423ef9a648bb66a763.svs,50cf88e9a33df0c0c8f9.svs,da18e7b9846e9d38034c.svs,d7a8af121d7d4f3fbf01.svs,2191a7aa287ce1d5dbc0.svs,c4b95da36e32993289cb.svs,72c93e042d0171a61012.svs,4eee7b944ad5e46c60ce.svs,3d3d04eca056556b0b26.svs,084383c18b9060880e82.svs,d37ab62158945f22deed.svs,deb768e5efb9d1dcbc13.svs
Best threshold: F1= 0.791042144242638 Threshold= 0.8100000000000005
Calculating F1 for test set of 7 files
Overall: 
TP: 4191 FP: 1395 FN:  1245 F1: 0.7604790419161677
Number of mitotic figures: 5436
Precision: 0.750 
Recall: 0.771


### And finally, the manual expert-labeled set with two stages

In [7]:

databasefile = 'databases/MITOS_WSI_CMC_MEL.sqlite'

resfile = 'results/trainval_RetinaNet-CMC-MEL-512sh-b1.pth-MEL-val-inference_results_boxes.p.bz2'
ident = f'MEL_2nd'

optimal_threshold[ident], F1scores, thrs = optimize_threshold(databasefile=databasefile, minthres=0.3, resfile=resfile)

resfile = 'results/trainval_2ndstage_RetinaNet-CMC-MEL-512sh-b1.pth-MELshort-val-inference_results_boxes.p.bz2'

F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, det_thres=optimal_threshold[ident])



Optimizing threshold for validation set of 14 files:  a8773be388e12df89edd.svs,460906c0b1fe17ea5354.svs,d0423ef9a648bb66a763.svs,50cf88e9a33df0c0c8f9.svs,da18e7b9846e9d38034c.svs,d7a8af121d7d4f3fbf01.svs,2191a7aa287ce1d5dbc0.svs,c4b95da36e32993289cb.svs,72c93e042d0171a61012.svs,4eee7b944ad5e46c60ce.svs,3d3d04eca056556b0b26.svs,084383c18b9060880e82.svs,d37ab62158945f22deed.svs,deb768e5efb9d1dcbc13.svs
Best threshold: F1= 0.7036853821419737 Threshold= 0.6300000000000003
Calculating F1 for test set of 14 files
Overall: 
TP: 7168 FP: 4776 FN:  980 F1: 0.7135178180370296
Number of mitotic figures: 8148
Precision: 0.600 
Recall: 0.880


### Print out table 2 in the manuscript

In [8]:
print('Single stage (RetinaNet, 512$\\times$512)'+f' & {F1_values["MEL"]:.3f} & {F1_values["ODAEL"]:.3f} & {F1_values["CODAEL"]:.3f} \\')
print(f'Dual stage (RetinaNet + ResNet-18) & {F1_values["MEL_2nd"]:.3f} & {F1_values["ODAEL_2nd"]:.3f} & {F1_values["CODAEL_2nd"]:.3f} \\')

Single stage (RetinaNet, 512$\times$512) & 0.684 & 0.705 & 0.726 \
Dual stage (RetinaNet + ResNet-18) & 0.714 & 0.760 & 0.793 \


# Transfer on human tissue

As described in the paper, we perform transfer tasks on human mammary carcinoma. For this, we detected mitoses on the publically available TUPAC16 set and its subsets. The
AMIDA13 set is the first 23 cases of the TUPAC16 set and was acquired with a different scanner (Aperio ScanScope), which makes the image acquisition parameters very similar to our image acquisition parameters and reduced the domain shift significantly.

Here, we test against the following conditions:

- AMIDA13: AMIDA13 complete set
- AMIDA13 test: AMIDA13 test set (last part of slides, used to compare against other approaches)
- TUPAC16 training set

We also relabeled the TUPAC16 set using almost the same methodology as in our paper and published this here:

- Bertram, C. A., Veta, M., Marzahl, C., Stathonikos, N., Maier, A., Klopfleisch, R., & Aubreville, M. (2020). Are pathologist-defined labels reproducible? Comparison of the TUPAC16 mitotic figure dataset with an alternative set of labels. In Interpretable and Annotation-Efficient Learning for Medical Image Computing (pp. 204-213). Springer, Cham. 
https://www.springerprofessional.de/are-pathologist-defined-labels-reproducible-comparison-of-the-tu/18445612

Find the complete dataset here:
https://github.com/DeepPathology/TUPAC16_AlternativeLabels

When refering to the alternative labels or AL, we refer to this alternative labels for the TUPAC16 set.




- Training on: CMC dataset
- Test on: TUPAC16 (original labels)
- Subset: AMIDA13 test set (Slides 13..23)

In [9]:

databasefile = 'databases/MITOS_WSI_CMC_CODAEL_TR_ROI.sqlite'

resfile = 'results/trainval_RetinaNet-CMC-CODAEL-512sh-b1.pth-CODAEL-val-inference_results_boxes.p.bz2'
ident = f'CODAEL_transfer'

optimal_threshold[ident], F1scores, thrs = optimize_threshold(databasefile=databasefile, minthres=0.3, resfile=resfile)

databasefile = 'databases/TUPAC_stitched.sqlite'
resfile = 'results/amidatest_2ndstage_CellClassifier_128px_CODAEL_b1.pthRetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Optimizing threshold for validation set of 14 files:  a8773be388e12df89edd.svs,460906c0b1fe17ea5354.svs,d0423ef9a648bb66a763.svs,50cf88e9a33df0c0c8f9.svs,da18e7b9846e9d38034c.svs,d7a8af121d7d4f3fbf01.svs,2191a7aa287ce1d5dbc0.svs,c4b95da36e32993289cb.svs,72c93e042d0171a61012.svs,4eee7b944ad5e46c60ce.svs,3d3d04eca056556b0b26.svs,084383c18b9060880e82.svs,d37ab62158945f22deed.svs,deb768e5efb9d1dcbc13.svs
Best threshold: F1= 0.7527488681131299 Threshold= 0.6300000000000003
Calculating F1 for test set of 11 files
Overall: 
TP: 362 FP: 270.0 FN:  171 F1: 0.6214592274678111
Number of mitotic figures: 533
Precision: 0.573 
Recall: 0.679


- Training on: CMC dataset
- Test on: TUPAC16 (alternative labels)
- Subset: AMIDA13 test set (Slides 13..23)

In [10]:
databasefile = 'databases/TUPAC_alternativeLabels_augmented_training.sqlite'
resfile = 'results/amidatest_2ndstage_CellClassifier_128px_CODAEL_b1.pthRetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 11 files
Overall: 
TP: 435 FP: 196 FN:  173 F1: 0.7021791767554479
Number of mitotic figures: 608
Precision: 0.689 
Recall: 0.715


- Training on: CMC dataset
- Test on: TUPAC16 (alternative labels)

In [11]:
databasefile = 'databases/TUPAC_alternativeLabels_augmented_training.sqlite'
resfile = 'results/tupactrain_2ndstage_CellClassifier_128px_CODAEL_b1.pthRetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 73 files
Overall: 
TP: 1189.0 FP: 581.0 FN:  810.0 F1: 0.6309365879543646
Number of mitotic figures: 1999
Precision: 0.672 
Recall: 0.595


- Training on: CMC dataset
- Test on: TUPAC16 (original labels)


In [12]:
databasefile = 'databases/TUPAC_stitched.sqlite'
resfile = 'results/tupactrain_2ndstage_CellClassifier_128px_CODAEL_b1.pthRetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 73 files
Overall: 
TP: 944.0 FP: 831.0 FN:  608.0 F1: 0.567478208596333
Number of mitotic figures: 1552
Precision: 0.532 
Recall: 0.608


## AMIDA13, complete set

- Training on: CMC dataset
- Test on: TUPAC16 (alternative labels)
- Subset: AMIDA13 complete set (Slides 1..23)

In [13]:
databasefile = 'databases/TUPAC_alternativeLabels_augmented_training.sqlite'
resfile = 'results/amida_2ndstage_CellClassifier_128px_CODAEL_b1.pthRetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 23 files
Overall: 
TP: 961 FP: 506 FN:  368 F1: 0.6874105865522174
Number of mitotic figures: 1329
Precision: 0.655 
Recall: 0.723


- Training on: CMC dataset
- Test on: TUPAC16 (original labels)
- Subset: AMIDA13 complete set (Slides 1..23)

In [14]:
databasefile = 'databases/TUPAC_stitched.sqlite'
resfile = 'results/amida_2ndstage_CellClassifier_128px_CODAEL_b1.pthRetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 23 files
Overall: 
TP: 749 FP: 722.0 FN:  334 F1: 0.586530931871574
Number of mitotic figures: 1083
Precision: 0.509 
Recall: 0.692


# RetinaNet only (single stage)

Now, for a comparison, only the first stage is evaluated.

- Training on: CMC dataset
- Test on: TUPAC16 (original labels)
- Subset: AMIDA13 complete set (Slides 1..23)

In [15]:
databasefile = 'databases/TUPAC_stitched.sqlite'
resfile = 'results/amida_RetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 23 files
Overall: 
TP: 479 FP: 314.0 FN:  604 F1: 0.5106609808102346
Number of mitotic figures: 1083
Precision: 0.604 
Recall: 0.442


- Training on: CMC dataset
- Test on: TUPAC16 (alternative labels)
- Subset: AMIDA13 complete set (Slides 1..23)

In [16]:
databasefile = 'databases/TUPAC_alternativeLabels_augmented_training.sqlite'
resfile = 'results/amida_RetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 23 files
Overall: 
TP: 581 FP: 213.0 FN:  748 F1: 0.5473386716910033
Number of mitotic figures: 1329
Precision: 0.732 
Recall: 0.437


- Training on: CMC dataset
- Test on: TUPAC16 (original labels)
- Subset: AMIDA13 test set (Slides 13..23)

In [17]:
databasefile = 'databases/TUPAC_stitched.sqlite'
resfile = 'results/amidatest_RetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 11 files
Overall: 
TP: 227 FP: 110.0 FN:  306 F1: 0.5218390804597701
Number of mitotic figures: 533
Precision: 0.674 
Recall: 0.426


- Training on: CMC dataset
- Test on: TUPAC16 (alternative labels)
- Subset: AMIDA13 test set (Slides 13..23)

In [18]:
databasefile = 'databases/TUPAC_alternativeLabels_augmented_training.sqlite'
resfile = 'results/amidatest_RetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 11 files
Overall: 
TP: 263 FP: 75 FN:  345 F1: 0.5560253699788583
Number of mitotic figures: 608
Precision: 0.778 
Recall: 0.433


- Training on: CMC dataset
- Test on: TUPAC16 (alternative labels)


In [19]:
databasefile = 'databases/TUPAC_alternativeLabels_augmented_training.sqlite'
resfile = 'results/tupactrain_RetinaNet-CMC-CODAEL-512sh-b1.pth-TUPAC_AL-val-inference_results_boxes.p.bz2'
F1_values[ident], individ = calculate_F1(databasefile=databasefile, resfile=resfile, hotclass=1, det_thres=optimal_threshold[ident])



Calculating F1 for test set of 73 files
Overall: 
TP: 822.0 FP: 310.0 FN:  1177.0 F1: 0.5250718620249122
Number of mitotic figures: 1999
Precision: 0.726 
Recall: 0.411


- Training on: CMC dataset
- Test on: TUPAC16 (original labels)
